# Required libraries

In [ ]:
!huggingface-cli login

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio

In [ ]:
!pip install peft

In [ ]:
!pip install langchain faiss-cpu sentence-transformers

In [ ]:
!pip install -U langchain-community

#Rag Database

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


# Data structure and Memory System

In [ ]:
import torch
import json
import os
from datetime import datetime # Import datetime
from typing import Dict, List
import gradio as gr

class SimpleDataManager:
    """Simple file-based storage for user data and recommendations"""

    def __init__(self, data_dir="wellbeing_data"):
        self.data_dir = data_dir
        os.makedirs(data_dir, exist_ok=True)
        os.makedirs(f"{data_dir}/users", exist_ok=True)
        os.makedirs(f"{self.data_dir}/recommendations", exist_ok=True) # Corrected path

    def save_user_week(self, user_id: str, week_data: Dict):
        """Save weekly data for a user"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            # Load existing data
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
            else:
                user_data = {"user_id": user_id, "weeks": []}

            # Add new week
            user_data["weeks"].append(week_data)

            # Keep only last 8 weeks
            user_data["weeks"] = user_data["weeks"][-8:]

            # Save
            with open(user_file, 'w') as f:
                json.dump(user_data, f, indent=2)

            print(f"✅ Saved week data for user {user_id}")

        except Exception as e:
            print(f"❌ Error saving user data: {str(e)}")

    def get_user_history(self, user_id: str) -> List[Dict]:
        """Get user's weekly history"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
                    history = user_data.get("weeks", [])
                    print(f"📊 Retrieved {len(history)} weeks of history for {user_id}")
                    return history
        except Exception as e:
            print(f"❌ Error loading user history: {str(e)}")

        return []

    def save_recommendation(self, user_id: str, week_start: str, recommendation: str):
        """Save LLM recommendation"""
        # Clean week_start for filename (remove invalid characters)
        clean_week = week_start.replace("/", "-").replace(":", "-")
        rec_file = f"{self.data_dir}/recommendations/{user_id}_{clean_week}.json"

        try:
            rec_data = {
                "user_id": user_id,
                "week_start": week_start,
                "recommendation": recommendation,
                "timestamp": datetime.now().isoformat()
            }

            with open(rec_file, 'w') as f:
                json.dump(rec_data, f, indent=2)

            print(f"✅ Saved recommendation for user {user_id}, week {week_start}")

        except Exception as e:
            print(f"❌ Error saving recommendation: {str(e)}")

    def get_last_recommendation(self, user_id: str) -> str:
        """Get user's last recommendation"""
        rec_dir = f"{self.data_dir}/recommendations"

        try:
            # Check if recommendations directory exists
            if not os.path.exists(rec_dir):
                return ""

            # Find latest recommendation file for this user
            user_files = [f for f in os.listdir(rec_dir) if f.startswith(f"{user_id}_") and f.endswith('.json')]

            if user_files:
                # Sort by date and get latest
                user_files.sort(reverse=True)
                latest_file = f"{rec_dir}/{user_files[0]}"

                with open(latest_file, 'r') as f:
                    rec_data = json.load(f)
                    recommendation = rec_data.get("recommendation", "")
                    print(f"📝 Retrieved last recommendation for {user_id} ({len(recommendation)} chars)")
                    return recommendation

        except Exception as e:
            print(f"❌ Error loading last recommendation: {str(e)}")

        return ""

    def get_user_stats(self, user_id: str) -> Dict:
        """Get basic stats about user's data (bonus method)"""
        history = self.get_user_history(user_id)

        if not history:
            return {"total_weeks": 0}

        return {
            "total_weeks": len(history),
            "first_week": history[0].get("week_start", "Unknown"),
            "latest_week": history[-1].get("week_start", "Unknown"),
            "avg_steps": sum(week.get("total_steps", 0) for week in history) // len(history),
            "avg_sleep": sum(week.get("avg_sleep", 0) for week in history) / len(history)
        }

#Well being LLM

In [ ]:
import torch
import re
from typing import Dict, List

class WellbeingLLM:
    """Improved LLM system for wellbeing recommendations with better prompt engineering"""

    def __init__(self, base_model_id: str, adapter_path: str, vectorstore_path: str = None):
        self.model = None
        self.tokenizer = None
        self.device = None
        self.vectorstore = None

        # Load vectorstore if path provided
        if vectorstore_path:
            self.load_vectorstore(vectorstore_path)

        self.load_model(base_model_id, adapter_path)

    def load_vectorstore(self, vectorstore_path: str):
        """Load FAISS vectorstore for RAG"""
        try:
            from langchain.vectorstores import FAISS
            from langchain.embeddings import HuggingFaceEmbeddings

            print("📚 Loading knowledge base...")
            embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

            self.vectorstore = FAISS.load_local(
                vectorstore_path,
                embeddings=embedding_model,
                allow_dangerous_deserialization=True
            )
            print("✅ Knowledge base loaded")

        except Exception as e:
            print(f"⚠️ Could not load vectorstore: {e}")
            self.vectorstore = None

    def load_model(self, base_model_id: str, adapter_path: str = None):
        """Load the LLM model with better error handling"""
        try:
            print("🤖 Loading LLM...")

            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
            from peft import PeftModel

            # Clear GPU memory and check device
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                print(f"🔍 GPU available: {torch.cuda.get_device_name()}")
            else:
                print("🔍 Using CPU")

            # Load tokenizer
            print("📝 Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer loaded")

            # Quantization config
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=False,
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            print("⚙️ Quantization config ready")

            # Load base model
            print("🧠 Loading base model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model_id,
                device_map="auto",
                torch_dtype=torch.float16,  # Faster than float16
                quantization_config=bnb_config,
                trust_remote_code=True,
                low_cpu_mem_usage=True,      # 🚀 NEW: Reduces CPU memory usage
                use_cache=False,             # 🚀 NEW: Faster loading
            )
            print("✅ Base model loaded")

            # Load fine-tuned adapter
            print("🎯 Loading fine-tuned adapter...")
            self.model = PeftModel.from_pretrained(self.model, adapter_path)
            self.model.eval()

            # Store device for later use
            self.device = next(self.model.parameters()).device
            print(f"✅ LLM loaded successfully on {self.device}")

        except Exception as e:
            print(f"❌ LLM loading failed: {str(e)}")
            import traceback
            traceback.print_exc()
            self.model = None
            self.tokenizer = None
            self.device = None




    def retrieve_relevant_context(self, current_week: Dict) -> Dict[str, str]:
        """Retrieve relevant information from vectorstore - IMPROVED"""
        if not self.vectorstore:
            return {}

        try:
            # Build targeted search queries
            total_steps = current_week.get('total_steps', 0)
            zone_minutes = current_week.get('zone_minutes', 0)
            avg_sleep = current_week.get('avg_sleep', 0)
            diet_type = current_week.get('preferences', {}).get('diet_type', 'No Preference')

            # Create category-specific queries
            category_contexts = {}

            # Food context
            if diet_type != 'No Preference':
                food_docs = self.vectorstore.similarity_search(f"{diet_type} nutrition guidelines", k=1)
                if food_docs:
                    category_contexts['food'] = food_docs[0].page_content[:150]

            # Exercise context
            if total_steps < 70000 or zone_minutes < 150:
                exercise_docs = self.vectorstore.similarity_search("exercise recommendations cardio fitness", k=1)
                if exercise_docs:
                    category_contexts['exercise'] = exercise_docs[0].page_content[:150]

            # Sleep context
            if avg_sleep < 7.5:
                sleep_docs = self.vectorstore.similarity_search("sleep hygiene improvement tips", k=1)
                if sleep_docs:
                    category_contexts['sleep'] = sleep_docs[0].page_content[:150]

            print(f"📚 Retrieved {len(category_contexts)} category-specific contexts")
            return category_contexts

        except Exception as e:
            print(f"⚠️ Error retrieving context: {e}")
            return {}

    def generate_recommendation(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
      """Generate personalized recommendations with single template enforcement"""

      if not self.model or not self.tokenizer:
          print("❌ No model loaded")
          return "Model not available. Please check model loading."

      max_retries = 3

      for attempt in range(max_retries):
          try:
              print(f"🔍 Attempt {attempt + 1} - Using single template strategy...")

              # Use the single master template
              prompt = self._build_master_template(current_week, user_history)
              print(f"🔍 Using master template ({len(prompt)} chars)")

              inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2000)
              inputs = {k: v.to(self.device) for k, v in inputs.items()}

              with torch.no_grad():
                  outputs = self.model.generate(
                      **inputs,
                      max_new_tokens=400,
                      min_new_tokens=150,
                      temperature=0.2,  # Low temperature for consistency
                      top_p=0.85,
                      do_sample=True,
                      repetition_penalty=1.15,
                      early_stopping=False,
                      pad_token_id=self.tokenizer.eos_token_id,
                      eos_token_id=self.tokenizer.eos_token_id,
                      use_cache=True
                  )

              full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
              response = full_response[len(prompt):].strip()

              # Clean response
              response = self._clean_response(response)

              # Validate structure
              if self._validate_template_structure(response):
                  print(f"✅ Template followed successfully on attempt {attempt + 1}")
                  return response
              else:
                  print(f"❌ Template validation failed on attempt {attempt + 1}")

          except Exception as e:
              print(f"❌ Generation failed on attempt {attempt + 1}: {str(e)}")

      # If all attempts fail, return the last attempt anyway (LLM-generated)
      print("⚠️ Returning best attempt even if template isn't perfect")
      return response if 'response' in locals() else "Unable to generate recommendation."

    def _build_master_template(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Single master template that forces LLM compliance"""

        # Extract user data
        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)
        diet_type = current_week.get('preferences', {}).get('diet_type', 'No Preference')
        allergies = current_week.get('preferences', {}).get('allergies', [])

        # Build progress context
        progress_info = self._get_progress_analysis(current_week, user_history)
        specific_needs = self._identify_specific_needs(current_week)

        # ADD THIS: Retrieve RAG context
        rag_contexts = self.retrieve_relevant_context(current_week)

        # Build context strings
        food_context = f"\nEXPERT KNOWLEDGE: {rag_contexts.get('food', '')}" if 'food' in rag_contexts else ""
        exercise_context = f"\nEXPERT KNOWLEDGE: {rag_contexts.get('exercise', '')}" if 'exercise' in rag_contexts else ""
        sleep_context = f"\nEXPERT KNOWLEDGE: {rag_contexts.get('sleep', '')}" if 'sleep' in rag_contexts else ""


        # The master template using forced completion technique
        prompt = f"""You are completing a wellness recommendation template. Fill in each section based on the user data provided.

USER DATA:
Steps: {total_steps:,}/70,000 weekly target
Cardio: {zone_minutes}/150 minutes weekly target
Sleep: {avg_sleep:.1f} hours nightly
Diet preference: {diet_type}
Allergies: {', '.join(allergies) if allergies else 'None'}

{progress_info}

AREAS NEEDING ATTENTION:
{specific_needs}
{food_context}
{exercise_context}
{sleep_context}

TEMPLATE TO COMPLETE:
You are a wellbeing coach AI that provides personalized health recommendations. Always structure your response in the exact format below, using the specified sections and subsections. Be specific, actionable, and evidence-based in your recommendations.
MANDATORY FORMAT:
1) Food Recommendation
   a) Overall Assessment
   b) Areas to Improve
   c) Meal and Food Suggestions

2) Physical Exercise
   a) Activity and Performance Analysis
   b) Zone Minutes and Intensity Feedback
   c) Strength Training Analysis
   d) Cardio Suggestions

3) Sleep and Social Well-being
   a) Sleep/Mood Analysis
   b) Suggestions

4) Overall Weekly Recommendation
   a) Summary
   b) Weekly Goals"""

        return prompt

    def _validate_template_structure(self, response: str) -> bool:
        """Validate the response follows the template structure"""

        # Check for main sections
        required_sections = [
            "## 1) Food",
            "## 2) Physical Exercise Suggestion",
            "## 3) Sleep and Social Well-being",
            "## 4) Overall Suggestion"
        ]

        sections_found = 0
        for section in required_sections:
            if section in response:
                sections_found += 1
            else:
                print(f"❌ Missing section: {section}")

        # Check for required bullet points in each section
        bullet_points_found = response.count('- Specific vegetables to try:')
        bullet_points_found += response.count('- Legumes and proteins:')
        bullet_points_found += response.count('- Meal suggestions:')
        bullet_points_found += response.count('- Cardio activities:')
        bullet_points_found += response.count('- Strength exercises:')
        bullet_points_found += response.count('- Sleep tips:')
        bullet_points_found += response.count('- Wellness activities:')
        bullet_points_found += response.count('- Top priorities:')
        bullet_points_found += response.count('- Weekly goals:')

        print(f"🔍 Found {sections_found}/4 sections and {bullet_points_found}/9 bullet points")

        # Consider valid if we have at least 3 sections and 6 bullet points
        return sections_found >= 3 and bullet_points_found >= 6

    # Alternative approach - Even more constrained template
    def _build_ultra_constrained_template(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Ultra-constrained template that gives LLM almost no choice"""

        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)
        diet_type = current_week.get('preferences', {}).get('diet_type', 'No Preference')

        prompt = f"""INSTRUCTION: Complete this exact template. Replace each [BLANK] with appropriate content. Do not change the structure, headings, or bullet point labels.

USER: {total_steps:,} steps, {zone_minutes} cardio minutes, {avg_sleep:.1f}h sleep, {diet_type} diet

## 1) Food
- Specific vegetables to try: [BLANK - list 3-4 vegetables]
- Legumes and proteins: [BLANK - list 2-3 proteins]
- Meal suggestions: [BLANK - list 2-3 meals]

## 2) Physical Exercise Suggestion
- Cardio activities: [BLANK - list 2-3 cardio exercises]
- Strength exercises: [BLANK - list 2-3 strength exercises]

## 3) Sleep and Social Well-being
- Sleep tips: [BLANK - list 2-3 sleep tips]
- Wellness activities: [BLANK - list 1-2 wellness activities]

## 4) Overall Suggestion
- Top priorities: [BLANK - list 2-3 priorities]
- Weekly goals: [BLANK - list 2-3 weekly goals]

COMPLETED TEMPLATE:

## 1) Food
- Specific vegetables to try:"""

        return prompt

    # Simplified generation function using the ultra-constrained approach
    def generate_with_ultra_constraint(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Use the most constrained template approach"""

        prompt = self._build_ultra_constrained_template(current_week, user_history)

        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1800)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=350,
                temperature=0.1,  # Very low temperature
                top_p=0.8,
                do_sample=True,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id
            )

        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = full_response[len(prompt):].strip()

        return self._clean_response(response)


    def _build_personalized_prompt(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Build a highly personalized prompt with STRICT formatting requirements including demographics"""

        # Extract comprehensive user data
        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)
        avg_mood = current_week.get('avg_mood', 0)
        exercise_sessions = current_week.get('exercise_sessions', 0)

        # Extract demographic data
        demographics = current_week.get('demographics', {})
        sex = demographics.get('sex', 'Not specified')
        age = demographics.get('age', 'Not specified')
        height = demographics.get('height', 'Not specified')
        weight = demographics.get('weight', 'Not specified')
        bmi = demographics.get('bmi', 'Not calculated')

        # Format demographic display
        height_display = f"{height} cm" if height != 'Not specified' else height
        weight_display = f"{weight} kg" if weight != 'Not specified' else weight
        bmi_display = f"{bmi}" if bmi != 'Not calculated' else bmi

        # Get preferences
        preferences = current_week.get('preferences', {})
        diet_type = preferences.get('diet_type', 'No Preference')
        allergies = preferences.get('allergies', [])
        cuisine_prefs = preferences.get('cuisine_preferences', [])

        # Get progress context
        progress_info = self._get_progress_analysis(current_week, user_history)
        specific_needs = self._identify_specific_needs(current_week)

        # Create STRICT formatting prompt with example including demographics
        prompt = f"""You are a wellness coach. You MUST follow this EXACT format. Do not deviate from this structure.

    USER PROFILE:
    Sex: {sex}
    Age: {age} years
    Height: {height_display}
    Weight: {weight_display}
    BMI: {bmi_display}

    CURRENT WEEK DATA:
    Steps: {total_steps:,} (goal: 70,000)
    Cardio: {zone_minutes} minutes (goal: 150)
    Workouts: {exercise_sessions} sessions
    Sleep: {avg_sleep:.1f}h per night
    Mood: {avg_mood:.1f}/10
    Diet: {diet_type}
    Allergies: {', '.join(allergies) if allergies else 'None'}

    {progress_info}

    SPECIFIC AREAS TO ADDRESS:
    {specific_needs}

    IMPORTANT: Tailor your recommendations based on the user's age ({age}), sex ({sex}), and BMI ({bmi_display}). Consider age-appropriate exercises, sex-specific nutritional needs, and BMI-related health considerations.

    EXAMPLE OUTPUT FORMAT (YOU MUST COPY THIS STRUCTURE EXACTLY):

    ## 1) Food
    - Specific vegetables to try: spinach, bell peppers, broccoli
    - Legumes and proteins: lentils, chickpeas, black beans
    - Meal suggestions: quinoa bowl with roasted vegetables, lentil curry with brown rice

    ## 2) Physical Exercise Suggestion
    - Cardio activities: brisk walking, cycling, swimming
    - Strength exercises: bodyweight squats, push-ups, planks

    ## 3) Sleep and Social Well-being
    - Sleep tips: maintain consistent bedtime, avoid screens 1 hour before sleep
    - Wellness activities: meditation, social walks with friends

    ## 4) Overall Suggestion
    - Top priorities: increase daily steps, improve sleep consistency
    - Weekly goals: walk 30 minutes daily, sleep by 10 PM

    YOUR TASK: Using the user's profile and data above, fill in this EXACT template structure. Consider their age, sex, height, weight, and BMI for personalized recommendations. Use the same headings, bullet points, and format. Do not add extra sections or change the structure.

    ## 1) Food
    - Specific vegetables to try: [list 3-4 vegetables, consider age-related nutritional needs]
    - Legumes and proteins: [list 2-3 protein sources considering sex, age, weight, and allergies]
    - Meal suggestions: [give 2-3 meal ideas for their diet type and demographic profile]

    ## 2) Physical Exercise Suggestion
    - Cardio activities: [list 2-3 activities appropriate for their age, sex, and fitness level]
    - Strength exercises: [list 2-3 exercises suitable for their demographic and BMI]

    ## 3) Sleep and Social Well-being
    - Sleep tips: [2-3 specific tips based on their sleep data and age]
    - Wellness activities: [1-2 activities appropriate for their age and lifestyle]

    ## 4) Overall Suggestion
    - Top priorities: [2-3 most important areas based on their demographic profile and data]
    - Weekly goals: [specific, measurable goals tailored to their age, sex, and current metrics]

    Response:"""

        return prompt


    def _identify_specific_needs(self, current_week: Dict) -> str:
        """Identify specific areas needing attention"""
        needs = []

        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)
        food_data = current_week.get('food_data', {})

        # Activity needs
        if total_steps < 70000:
            deficit = 70000 - total_steps
            needs.append(f"ACTIVITY: Add {deficit:,} more steps ({deficit//7:,} per day)")

        if zone_minutes < 150:
            deficit = 150 - zone_minutes
            needs.append(f"CARDIO: Add {deficit} more zone minutes")

        # Sleep needs
        if avg_sleep < 7:
            needs.append(f"SLEEP: Increase by {7 - avg_sleep:.1f} hours nightly")

        # Nutrition needs
        water = food_data.get('water_liters', 0)
        # if water < 14:
        #     needs.append(f"HYDRATION: Drink {14 - water:.1f}L more water weekly")

        vegetables = food_data.get('vegetables_grams', 0)
        # if vegetables < 1750:
        #     needs.append(f"NUTRITION: Add {1750 - vegetables:.0f}g more vegetables weekly")

        return '\n'.join(needs) if needs else "Maintaining current healthy patterns"

    def _get_progress_analysis(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Analyze progress for personalized feedback"""
        if len(user_history) < 2:
            return "PROGRESS: First week tracked"

        prev_week = user_history[-2]

        changes = []

        # Step progress
        step_change = current_week.get('total_steps', 0) - prev_week.get('total_steps', 0)
        if step_change > 0:
            changes.append(f"Steps IMPROVED by {step_change:,}")
        elif step_change < 0:
            changes.append(f"Steps DECREASED by {abs(step_change):,}")

        # Zone minutes progress
        zone_change = current_week.get('zone_minutes', 0) - prev_week.get('zone_minutes', 0)
        if zone_change > 0:
            changes.append(f"Cardio IMPROVED by {zone_change} minutes")
        elif zone_change < 0:
            changes.append(f"Cardio DECREASED by {abs(zone_change)} minutes")

        # Sleep progress
        sleep_change = current_week.get('avg_sleep', 0) - prev_week.get('avg_sleep', 0)
        if sleep_change > 0.2:
            changes.append(f"Sleep IMPROVED by {sleep_change:.1f}h")
        elif sleep_change < -0.2:
            changes.append(f"Sleep DECREASED by {abs(sleep_change):.1f}h")

        progress_text = f"PROGRESS FROM LAST WEEK:\n{chr(10).join(changes)}" if changes else "PROGRESS: Similar to last week"
        return progress_text

    def _has_minimum_structure(self, response: str) -> bool:
        """Check if response has minimum required structure"""
        required_sections = ["1) Food", "2) Physical", "3) Sleep", "4) Overall"]
        found_sections = sum(1 for section in required_sections if section in response)
        return found_sections >= 3  # At least 3 out of 4 sections



    def _build_improved_prompt(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
        """Build a much cleaner, more focused prompt"""

        # Extract key data
        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)
        diet_type = current_week.get('preferences', {}).get('diet_type', 'No Preference')
        allergies = current_week.get('preferences', {}).get('allergies', [])

        # Get focused RAG context
        rag_contexts = self.retrieve_relevant_context(current_week)

        # Build clean context strings
        food_context = f"Expert guidance: {rag_contexts.get('food', '')}" if 'food' in rag_contexts else ""
        exercise_context = f"Expert guidance: {rag_contexts.get('exercise', '')}" if 'exercise' in rag_contexts else ""
        sleep_context = f"Expert guidance: {rag_contexts.get('sleep', '')}" if 'sleep' in rag_contexts else ""

        # Build allergy warnings
        allergy_warning = f"CRITICAL: Avoid all {', '.join(allergies)} - user is allergic!" if allergies else ""

        # Create focused prompt
        prompt = f"""Task: Provide health recommendations in exactly 4 categories.

User Data:
- Weekly steps: {total_steps:,}
- Cardio minutes: {zone_minutes}
- Sleep average: {avg_sleep:.1f}h
- Diet preference: {diet_type}
{allergy_warning}

{food_context}
{exercise_context}
{sleep_context}

Format (use exactly these headings):

## 1) Food
{f"For {diet_type} diet, recommend" if diet_type != 'No Preference' else "Recommend"} specific foods and meals.

## 2) Physical Exercise Suggestion
Based on {total_steps:,} steps and {zone_minutes} cardio minutes, suggest exercise.

## 3) Sleep and Social Well-being
With {avg_sleep:.1f}h sleep, provide sleep and wellness tips.

## 4) Overall Suggestion
Prioritize the most important improvement area.

Response:"""

        return prompt

    def _validate_exact_format(self, response: str) -> bool:
        """Check if response has exact required format"""
        required_sections = [
            "## 1) Food",
            "## 2) Physical Exercise Suggestion",
            "## 3) Sleep and Social Well-being",
            "## 4) Overall Suggestion"
        ]

        for section in required_sections:
            if section not in response:
                print(f"❌ Missing section: {section}")
                return False

        # Check for mixing of content (basic check)
        sections = response.split('##')
        if len(sections) < 5:  # Should have intro + 4 sections
            print("❌ Not enough sections found")
            return False

        return True

    def _clean_response(self, response: str) -> str:
        """Clean the generated response"""
        # Remove unwanted prefixes
        response = re.sub(r'^(Response:|Recommendations?:)', '', response, flags=re.IGNORECASE)
        response = response.strip()

        # Remove excessive newlines
        response = re.sub(r'\n{3,}', '\n\n', response)

        # Remove repetitive phrases
        response = re.sub(r'(Based on your.*?)\1+', r'\1', response, flags=re.IGNORECASE)

        return response.strip()




    def _validate_allergy_safety(self, response: str, current_week: Dict) -> bool:
        """Enhanced allergy safety validation"""
        preferences = current_week.get('preferences', {})
        allergies = preferences.get('allergies', [])

        if not allergies:
            return True

        response_lower = response.lower()

        # Enhanced allergy keyword mapping
        allergy_keywords = {
            'Nuts (Tree nuts)': ['nuts', 'almond', 'walnut', 'cashew', 'pecan', 'hazelnut', 'pistachio', 'macadamia'],
            'Peanuts': ['peanut', 'peanut butter', 'groundnut'],
            'Shellfish': ['shrimp', 'crab', 'lobster', 'shellfish', 'prawns', 'crayfish'],
            'Fish': ['salmon', 'tuna', 'fish', 'cod', 'mackerel', 'sardine'],
            'Milk/Dairy': ['milk', 'cheese', 'yogurt', 'dairy', 'butter', 'cream', 'whey', 'casein'],
            'Eggs': ['eggs', 'egg', 'mayonnaise'],
            'Soy': ['soy', 'tofu', 'soybean', 'miso', 'tempeh', 'edamame'],
            'Wheat/Gluten': ['wheat', 'gluten', 'bread', 'pasta', 'flour', 'barley', 'rye'],
            'Sesame': ['sesame', 'tahini', 'sesame oil']
        }

        for allergy in allergies:
            if allergy in allergy_keywords:
                for keyword in allergy_keywords[allergy]:
                    if keyword in response_lower:
                        print(f"⚠️ ALLERGY RISK: Found '{keyword}' but user allergic to {allergy}")
                        return False

        return True



    def generate_with_template(self, current_week: Dict) -> str:
        """Generate using templates with LLM filling blanks"""

        template = """## 1) Food
Based on your {diet_type} diet, {food_advice}

## 2) Physical Exercise Suggestion
With {steps} steps, {exercise_advice}

## 3) Sleep and Social Well-being
{sleep_advice}

## 4) Overall Suggestion
{overall_advice}"""

# Well being system

In [ ]:
class WellbeingSystem:
    """Main system that combines data management and LLM"""

    def __init__(self, base_model_id: str, adapter_path: str, vectorstore_path: str = None):
        self.data_manager = SimpleDataManager()
        self.llm = WellbeingLLM(base_model_id, adapter_path, vectorstore_path) # Pass vectorstore_path to LLM
        print("🎯 WellbeingSystem initialized")

    def analyze_and_recommend(self, user_id: str, week_data: Dict) -> str:
        """Main function: analyze weekly data and generate recommendations"""

        print(f"📊 Analyzing data for user: {user_id}")

        try:
            # Get user history and last recommendation
            user_history = self.data_manager.get_user_history(user_id)
            last_recommendation = self.data_manager.get_last_recommendation(user_id)

            print(f"📈 Found {len(user_history)} weeks of history")
            if last_recommendation:
                print(f"📝 Previous recommendation found ({len(last_recommendation)} chars)")

            # Process current week data
            processed_week = self._process_week_data(week_data)

            # Show what data is being sent to LLM
            print(f"📊 Processed data: {processed_week['total_steps']:,} steps, {processed_week['zone_minutes']} zone mins, {processed_week['avg_sleep']:.1f}h sleep")

            # Generate recommendation using LLM
            recommendation = self.llm.generate_recommendation(
                processed_week, user_history, last_recommendation
            )

            # Save data (with error handling)
            try:
                self.data_manager.save_user_week(user_id, processed_week)
                self.data_manager.save_recommendation(user_id, processed_week['week_start'], recommendation)
                print(f"💾 Data saved successfully")
            except Exception as save_error:
                print(f"⚠️ Warning: Could not save data: {save_error}")
                # Continue anyway - return the recommendation even if saving fails

            print(f"✅ Generated recommendation for {user_id} ({len(recommendation)} chars)")
            return recommendation

        except Exception as e:
            print(f"❌ Error in analyze_and_recommend: {str(e)}")
            import traceback
            traceback.print_exc()
            return f"Analysis failed: {str(e)}. Please check your input data."

    def _process_week_data(self, week_data: Dict) -> Dict:
        """Process and standardize weekly data with better validation"""

        try:
            # Parse sleep and mood data with validation
            sleep_hours = week_data.get('sleep_hours', [])
            mood_scores = week_data.get('mood_scores', [])

            # Handle string input (comma-separated values)
            if isinstance(sleep_hours, str):
                try:
                    sleep_hours = [float(x.strip()) for x in sleep_hours.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid sleep hours format, using default")
                    sleep_hours = []

            if isinstance(mood_scores, str):
                try:
                    mood_scores = [float(x.strip()) for x in mood_scores.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid mood scores format, using default")
                    mood_scores = []

            # Calculate averages with validation
            avg_sleep = sum(sleep_hours) / len(sleep_hours) if sleep_hours else 0
            avg_mood = sum(mood_scores) / len(mood_scores) if mood_scores else 0

            # Enhanced food data processing
            food_data = self._process_food_data(week_data)

            processed_data = {
                'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                'total_steps': max(0, int(week_data.get('total_steps', 0))), # Ensure int and non-negative
                'zone_minutes': max(0, int(week_data.get('zone_minutes', 0))), # Ensure int and non-negative
                'exercise_sessions': max(0, int(week_data.get('exercise_sessions', 0))), # Ensure int and non-negative
                'avg_sleep': round(avg_sleep, 1),
                'avg_mood': round(avg_mood, 1),
                'food_data': food_data,
                'timestamp': datetime.now().isoformat()
            }

            # Include preferences and targets if present in the input week_data
            if 'preferences' in week_data:
                processed_data['preferences'] = week_data['preferences']
            if 'targets' in week_data:
                processed_data['targets'] = week_data['targets']


            return processed_data

        except Exception as e:
            print(f"❌ Error processing week data: {str(e)}")
            # Return empty or default data structure on error
            return {
                 'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                 'total_steps': 0,
                 'zone_minutes': 0,
                 'exercise_sessions': 0,
                 'avg_sleep': 0.0,
                 'avg_mood': 0.0,
                 'food_data': {},
                 'timestamp': datetime.now().isoformat(),
                 'processing_error': str(e)
            }


    def _process_food_data(self, week_data: Dict) -> Dict:
        """Process food data handling both old and new formats"""

        food_data = {}

        # Check for new format first (with units)
        unit_fields = ['dairy_liters', 'water_liters', 'legumes_grams', 'meat_grams',
                      'fruits_grams', 'vegetables_grams', 'grains_grams', 'nuts_seeds_grams']

        has_unit_format = any(field in week_data for field in unit_fields)

        if has_unit_format:
            print("📊 Using new format (with units)")
            # New format with units
            for field in unit_fields:
                value = week_data.get(field, 0)
                try:
                    food_data[field] = max(0, float(value))  # Ensure non-negative
                except (ValueError, TypeError):
                    food_data[field] = 0
                    print(f"⚠️ Invalid {field} value, using 0")
        else:
            print("📊 Using old format (servings)")
            # Old format (for backward compatibility)
            old_fields = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']
            for field in old_fields:
                if field in week_data:
                    try:
                        food_data[field] = max(0, float(week_data[field]))
                    except (ValueError, TypeError):
                        food_data[field] = 0

        return food_data

    def get_user_progress(self, user_id: str) -> Dict:
        """Get user's progress over time with enhanced analysis"""

        try:
            history = self.data_manager.get_user_history(user_id)

            if len(history) < 1:
                return {"message": "No data available for this user"}

            if len(history) < 2:
                return {
                    "message": "Need at least 2 weeks of data to show progress",
                    "weeks_tracked": len(history),
                    "latest_week": history[0] if history else None
                }

            # Compare latest vs previous week
            current = history[-1]
            previous = history[-2]

            # Calculate changes
            progress = {
                'weeks_tracked': len(history),
                'steps_change': current.get('total_steps', 0) - previous.get('total_steps', 0),
                'zone_change': current.get('zone_minutes', 0) - previous.get('zone_minutes', 0),
                'sleep_change': round(current.get('avg_sleep', 0) - previous.get('avg_sleep', 0), 1),
                'mood_change': round(current.get('avg_mood', 0) - previous.get('avg_mood', 0), 1),
                'latest_week': current,
                'previous_week': previous
            }

            # Add trend analysis for longer history
            if len(history) >= 4:
                # Calculate 4-week averages for trends
                recent_4_weeks = history[-4:]
                avg_steps_recent = sum(w.get('total_steps', 0) for w in recent_4_weeks) / 4
                avg_sleep_recent = sum(w.get('avg_sleep', 0) for w in recent_4_weeks) / 4

                progress['trends'] = {
                    'avg_steps_4weeks': round(avg_steps_recent, 0),
                    'avg_sleep_4weeks': round(avg_sleep_recent, 1),
                    'consistency_score': self._calculate_consistency_score(recent_4_weeks)
                }

            return progress

        except Exception as e:
            print(f"❌ Error getting user progress: {str(e)}")
            return {"error": f"Could not retrieve progress: {str(e)}"}

    def _calculate_consistency_score(self, weeks: List[Dict]) -> float:
        """Calculate a consistency score based on week-to-week variation"""
        if len(weeks) < 2:
            return 0.0

        try:
            # Calculate coefficient of variation for steps (lower = more consistent)
            steps = [w.get('total_steps', 0) for w in weeks]
            if all(s == 0 for s in steps):
                return 0.0

            mean_steps = sum(steps) / len(steps)
            if mean_steps == 0:
                return 0.0

            variance = sum((s - mean_steps) ** 2 for s in steps) / len(steps)
            cv = (variance ** 0.5) / mean_steps

            # Convert to score (0-10, where 10 = most consistent)
            consistency_score = max(0, 10 - (cv * 10))
            return round(consistency_score, 1)

        except Exception:
            return 0.0



# Well being app

In [ ]:
import gradio as gr
import json
import os
from datetime import datetime
from typing import Dict, List

class WellbeingApp:
    """Complete Gradio interface with targets, history, and preferences"""

    def __init__(self, base_model_id: str, adapter_path: str):
      print("🚀 Initializing Complete WellbeingApp...")
      self.wellbeing_system = WellbeingSystem(
          base_model_id,
          adapter_path,
          vectorstore_path="/content/drive/MyDrive/rag_index"
      )

      # Default targets (same as before)
      self.default_targets = {
          'weekly_steps': 70000,
          'weekly_zone_minutes': 150,
          'daily_sleep_hours': 8.0,
          'weekly_water_liters': 14.0,
          'weekly_fruits_grams': 2100,
          'weekly_vegetables_grams': 3500,
          'weekly_exercise_sessions': 4
      }

    def create_wellbeing_app(self):
        """Create comprehensive Gradio interface with tabs"""

        with gr.Blocks(title="🏥 Complete Wellbeing LLM", theme=gr.themes.Soft()) as demo:

            gr.Markdown("# 🏥 Complete AI Wellbeing System")
            gr.Markdown("### Personalized health recommendations with custom targets and progress tracking")

            # Create tabs for different sections
            with gr.Tabs():

                # TAB 1: MAIN ANALYSIS
                with gr.TabItem("🤖 Get Recommendations", elem_id="main-tab"):
                    self._create_main_analysis_tab()

                # TAB 2: TARGETS CUSTOMIZATION
                with gr.TabItem("🎯 Customize Targets", elem_id="targets-tab"):
                    self._create_targets_tab()

                # TAB 3: HISTORY REVIEW
                with gr.TabItem("📈 Progress History", elem_id="history-tab"):
                    self._create_history_tab()

                # TAB 4: SYSTEM INFO
                # with gr.TabItem("ℹ️ System Info", elem_id="info-tab"):
                #     self._create_info_tab()

        return demo

    def _create_main_analysis_tab(self):
      """Create the main analysis tab with fixed component handling"""

      with gr.Row():
          with gr.Column():

              gr.Markdown("### 🤖 AI Enhancement Options")
              # User Info Section
              gr.Markdown("### 👤 User Information")
              user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Unique identifier for tracking your progress")
              week_start = gr.Textbox(label="📅 Week Start Date", value="2024-01-15", info="Format: YYYY-MM-DD")

              # Personal Details - FIXED: These were not being collected properly
              sex = gr.Dropdown(
                  label="⚧️ Sex",
                  choices=["Male", "Female", "Other", "Prefer not to say"],
                  value="Male",
                  info="Biological sex for health calculations"
              )

              age = gr.Number(
                  label="🎂 Age",
                  value=25,
                  minimum=13,
                  maximum=120,
                  step=1,
                  info="Age in years"
              )

              weight = gr.Number(
                  label="⚖️ Weight (kg)",
                  value=70.0,
                  minimum=30.0,
                  maximum=300.0,
                  step=0.1,
                  info="Current weight in kilograms"
              )

              height = gr.Number(
                  label="📏 Height (cm)",
                  value=175.0,
                  minimum=100.0,
                  maximum=250.0,
                  step=0.1,
                  info="Height in centimeters"
              )

              # Preferences Section
              gr.Markdown("### 🍽️ Meal Preferences & Dietary Restrictions")
              with gr.Row():
                  diet_type = gr.Dropdown(
                      label="🥗 Diet Type",
                      choices=[
                          "No Preference", "Vegetarian", "Vegan", "Pescatarian",
                          "Keto", "Paleo", "Mediterranean", "Low Carb", "Gluten Free", "Dairy Free"
                      ],
                      value="No Preference",
                      info="Select your primary dietary preference"
                  )

              allergies = gr.CheckboxGroup(
                  label="⚠️ Food Allergies & Intolerances",
                  choices=[
                      "Nuts (Tree nuts)", "Peanuts", "Shellfish", "Fish",
                      "Milk/Dairy", "Eggs", "Soy", "Wheat/Gluten",
                      "Sesame", "Lactose Intolerant", "Other"
                  ],
                  info="⚠️ IMPORTANT: Select all allergies and intolerances"
              )

              other_allergies = gr.Textbox(
                  label="🚨 Other Allergies/Restrictions",
                  placeholder="e.g., tomatoes, specific medications, religious restrictions...",
                  info="Specify any other dietary restrictions or allergies"
              )

              # Activity Section
              gr.Markdown("### 🏃‍♂️ Physical Activity")
              total_steps = gr.Number(label="Total Steps (Week)", value=65000, info="Your current target will be shown")
              zone_minutes = gr.Number(label="Zone Minutes (Week)", value=120, info="Your current target will be shown")
              exercise_sessions = gr.Number(label="Exercise Sessions", value=3, info="Your current target will be shown")

              # Sleep & Mood Section
              gr.Markdown("### 😴 Sleep & Mood")
              sleep_hours = gr.Textbox(
                  label="Sleep Hours (7 days)",
                  value="7.5,6.8,8.2,7.0,6.5,8.5,7.8",
                  info="Enter daily sleep hours separated by commas"
              )
              mood_scores = gr.Textbox(
                  label="Mood Scores (7 days)",
                  value="4,3.5,5,3,4,2.5,5",
                  info="Rate your daily mood from 1-5, separated by commas"
              )

              # Food Section
              gr.Markdown("### 🍎 Weekly Nutrition Intake")
              gr.Markdown("*Enter your total consumption for the entire week*")

              with gr.Row():
                  dairy = gr.Number(label="🥛 Dairy (liters)", value=3.5, info="Milk, yogurt, cheese")
                  water_liters = gr.Number(label="💧 Water (liters)", value=48, info="Your current target will be shown")

              with gr.Row():
                  fruits = gr.Number(label="🍎 Fruits (grams)", value=2100, info="Your current target will be shown")
                  vegetables = gr.Number(label="🥬 Vegetables (grams)", value=3500, info="Your current target will be shown")

              with gr.Row():
                  legumes = gr.Number(label="🫘 Legumes (grams)", value=350, info="Beans, lentils, peas")
                  nuts_seeds = gr.Number(label="🥜 Nuts/Seeds (grams)", value=210, info="Almonds, walnuts, chia seeds")

              with gr.Row():
                  meat = gr.Number(label="🍗 Meat (grams)", value=700, info="Chicken, beef, fish")
                  grains = gr.Number(label="🌾 Grains (grams)", value=2800, info="Rice, bread, pasta, oats")

              # Action buttons
              with gr.Row():
                  analyze_btn = gr.Button("🤖 Get AI Recommendations", variant="primary", size="lg")
                  load_targets_btn = gr.Button("📊 Load My Targets", variant="secondary")

          with gr.Column():
              main_output = gr.Markdown("""
  ## 📋 How to Use

  1. **Enter your User ID** - This tracks your progress over time
  2. **Set your personal details** - Age, sex, weight, height for personalized calculations
  3. **Set your dietary preferences** - Diet type, allergies, meal preferences
  4. **⚠️ CRITICAL: Select all allergies** - Ensures safe recommendations
  5. **Fill in your weekly data** - Activity, sleep, mood, nutrition
  6. **Click 'Get AI Recommendations'** - Receive personalized advice

  ### 🎯 Default Weekly Targets:
  - **Steps:** 70,000+ (10,000 daily)
  - **Zone Minutes:** 150+ (cardio exercise)
  - **Sleep:** 8 hours nightly
  - **Water:** 14+ liters (2L daily)
  - **Fruits:** 2100+ grams
  - **Vegetables:** 3500+ grams
  - **Exercise Sessions:** 4+

  *Go to 'Customize Targets' tab to modify these targets for your personal goals.*

  ### 🔒 Privacy & Safety:
  - All data stored locally on your device
  - AI recommendations respect your dietary restrictions
  - Always consult healthcare providers for medical advice
  """)

      # FIXED: Store components with correct order including demographics
      self.main_components = {
          'user_id': user_id,
          'week_start': week_start,
          'sex': sex,           # ADDED: Missing demographic fields
          'age': age,           # ADDED
          'weight': weight,     # ADDED
          'height': height,     # ADDED
          'total_steps': total_steps,
          'zone_minutes': zone_minutes,
          'exercise_sessions': exercise_sessions,
          'sleep_hours': sleep_hours,
          'mood_scores': mood_scores,
          'dairy_liters': dairy,
          'legumes_grams': legumes,
          'meat_grams': meat,
          'fruits_grams': fruits,
          'vegetables_grams': vegetables,
          'grains_grams': grains,
          'nuts_seeds_grams': nuts_seeds,
          'water_liters': water_liters,
          'diet_type': diet_type,
          'allergies': allergies,
          'other_allergies': other_allergies,
          'output': main_output
      }

      # FIXED: Connect button with correct inputs order
      analyze_btn.click(
          fn=self.analyze_wellbeing_with_preferences,
          inputs=[
              user_id, week_start, sex, age, weight, height,  # Demographics in correct order
              total_steps, zone_minutes, exercise_sessions,
              sleep_hours, mood_scores, dairy, legumes, meat,
              fruits, vegetables, grains, nuts_seeds, water_liters,
              diet_type, allergies, other_allergies
          ],
          outputs=[main_output]
      )

      load_targets_btn.click(
          fn=self.load_user_targets,
          inputs=[user_id],
          outputs=[main_output]
      )

    def analyze_wellbeing_with_preferences(self, user_id, week_start, sex, age, weight, height,
                                     total_steps, zone_minutes, exercise_sessions,
                                     sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
                                     fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
                                     diet_type, allergies, other_allergies):
      """Enhanced analysis with agent support, custom targets, and user demographics"""

      try:
          # Input validation
          if not user_id or not user_id.strip():
              return "❌ Error: Please enter a valid User ID"

          if not week_start:
              return "❌ Error: Please enter a week start date"

          # Validate demographic data
          if age is not None and (age < 13 or age > 120):
              return "❌ Error: Please enter a valid age (13-120 years)"

          if weight is not None and (weight < 30 or weight > 300):
              return "❌ Error: Please enter a valid weight (30-300 kg)"

          if height is not None and (height < 100 or height > 250):
              return "❌ Error: Please enter a valid height (100-250 cm)"

          # Load user's custom targets
          user_targets = self.get_user_targets(user_id.strip())

          # Calculate BMI if weight and height are provided
          bmi = None
          if weight and height:
              height_m = height / 100  # Convert cm to meters
              bmi = round(weight / (height_m ** 2), 1)

          # Prepare enhanced data with demographics
          week_data = {
              'user_id': user_id.strip(),
              'week_start': week_start,
              'demographics': {
                  'sex': sex,
                  'age': int(age) if age else None,
                  'weight': float(weight) if weight else None,
                  'height': float(height) if height else None,
                  'bmi': bmi
              },
              'total_steps': max(0, int(total_steps)) if total_steps else 0,
              'zone_minutes': max(0, int(zone_minutes)) if zone_minutes else 0,
              'exercise_sessions': max(0, int(exercise_sessions)) if exercise_sessions else 0,
              'sleep_hours': sleep_hours,
              'mood_scores': mood_scores,
              'dairy_liters': max(0, float(dairy_liters)) if dairy_liters else 0,
              'legumes_grams': max(0, float(legumes_grams)) if legumes_grams else 0,
              'meat_grams': max(0, float(meat_grams)) if meat_grams else 0,
              'fruits_grams': max(0, float(fruits_grams)) if fruits_grams else 0,
              'vegetables_grams': max(0, float(vegetables_grams)) if vegetables_grams else 0,
              'grains_grams': max(0, float(grains_grams)) if grains_grams else 0,
              'nuts_seeds_grams': max(0, float(nuts_seeds_grams)) if nuts_seeds_grams else 0,
              'water_liters': max(0, float(water_liters)) if water_liters else 0,
              'preferences': {
                  'diet_type': diet_type,
                  'allergies': allergies if allergies else [],
                  'other_allergies': other_allergies.strip() if other_allergies else ""
              },
              'targets': user_targets
          }

          recommendation = self.wellbeing_system.analyze_and_recommend(user_id.strip(), week_data)

          # Enhanced output with custom targets and demographics
          output = self._format_analysis_output_with_targets(week_data, recommendation, user_targets)

          return output

      except Exception as e:
          print(f"❌ Analysis error: {str(e)}")
          import traceback
          traceback.print_exc()
          return f"❌ Error: {str(e)}\n\nPlease check your input format and try again."


    def _format_food_data(self, food_data: Dict) -> str:
      """Format food data with proper units"""
      if not food_data:
          return "• No food data provided"

      food_lines = []
      unit_mapping = {
          'dairy_liters': ('Dairy', 'L'),
          'water_liters': ('Water', 'L'),
          'legumes_grams': ('Legumes', 'g'),
          'meat_grams': ('Meat', 'g'),
          'fruits_grams': ('Fruits', 'g'),
          'vegetables_grams': ('Vegetables', 'g'),
          'grains_grams': ('Grains', 'g'),
          'nuts_seeds_grams': ('Nuts/Seeds', 'g')
      }

      for key, amount in food_data.items():
          if amount > 0 and key in unit_mapping:
              name, unit = unit_mapping[key]
              food_lines.append(f"• {name}: {amount:.0f}{unit}")
          elif amount > 0:
              clean_name = key.replace('_', ' ').title()
              food_lines.append(f"• {clean_name}: {amount}")

      return '\n'.join(food_lines) if food_lines else "• No significant food intake recorded"

    def _build_progress_context(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Build progress context from user history"""
        if len(user_history) < 2:
            return "PROGRESS CONTEXT:\n• First week or limited history available"

        prev_week = user_history[-2]

        step_change = current_week.get('total_steps', 0) - prev_week.get('total_steps', 0)
        zone_change = current_week.get('zone_minutes', 0) - prev_week.get('zone_minutes', 0)
        sleep_change = current_week.get('avg_sleep', 0) - prev_week.get('avg_sleep', 0)

        context = f"""PROGRESS FROM LAST WEEK:
- Steps: {step_change:+,} change
- Zone minutes: {zone_change:+} change
- Sleep: {sleep_change:+.1f} hours change
- Total weeks tracked: {len(user_history)}"""

        return context

    def _build_preferences_context(self, preferences: Dict) -> str:
        """Build dietary preferences context for the LLM"""
        if not preferences:
            return "DIETARY PREFERENCES:\n• No specific preferences provided"

        context_lines = ["DIETARY PREFERENCES & RESTRICTIONS:"]

        diet_type = preferences.get('diet_type', 'No Preference')
        if diet_type != 'No Preference':
            context_lines.append(f"• Diet Type: {diet_type}")

        allergies = preferences.get('allergies', [])
        other_allergies = preferences.get('other_allergies', '')

        if allergies or other_allergies:
            context_lines.append("• ⚠️ ALLERGIES/RESTRICTIONS (MUST AVOID):")
            for allergy in allergies:
                context_lines.append(f"  - {allergy}")
            if other_allergies:
                context_lines.append(f"  - {other_allergies}")




        return '\n'.join(context_lines)
    def _create_targets_tab(self):
        """Create the targets customization tab"""

        gr.Markdown("## 🎯 Customize Your Personal Health Targets")
        gr.Markdown("### Set personalized goals based on your fitness level, health conditions, and personal objectives")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                target_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter the user ID to customize targets for")

                gr.Markdown("### 🏃‍♂️ Activity Targets")
                target_weekly_steps = gr.Number(
                    label="Weekly Steps Target",
                    value=70000,
                    info="Default: 70,000 (10,000 daily). Adjust based on fitness level."
                )
                target_zone_minutes = gr.Number(
                    label="Weekly Zone Minutes Target",
                    value=150,
                    info="Default: 150 minutes. WHO recommends 150+ minutes moderate exercise."
                )
                target_exercise_sessions = gr.Number(
                    label="Weekly Exercise Sessions Target",
                    value=4,
                    info="Default: 4 sessions. Include both cardio and strength training."
                )

                gr.Markdown("### 😴 Sleep & Recovery Targets")
                target_daily_sleep = gr.Number(
                    label="Daily Sleep Hours Target",
                    value=8.0,
                    info="Default: 8 hours. Range: 7-9 hours for most adults."
                )

                with gr.Row():
                    save_targets_btn = gr.Button("💾 Save My Targets", variant="primary", size="lg")
                    load_current_targets_btn = gr.Button("📊 Load Current Targets", variant="secondary")
                    reset_defaults_btn = gr.Button("🔄 Reset to Defaults", variant="secondary")


        # Store target components
        self.target_components = {
            'user_id': target_user_id,
            'weekly_steps': target_weekly_steps,
            'zone_minutes': target_zone_minutes,
            'exercise_sessions': target_exercise_sessions,
            'daily_sleep': target_daily_sleep,
        }

        # Connect buttons
        save_targets_btn.click(
            fn=self.save_user_targets,
            inputs=list(self.target_components.values())[:-1],
            # outputs=[targets_output]
        )

        load_current_targets_btn.click(
            fn=self.load_user_targets_display,
            inputs=[target_user_id],
            outputs=list(self.target_components.values()) # Update all target number boxes and output
        )

        reset_defaults_btn.click(
            fn=self.reset_to_defaults,
            outputs=list(self.target_components.values())
        )


    def _create_history_tab(self):
        """Create the history review tab"""

        gr.Markdown("## 📈 Progress History & Analytics")
        gr.Markdown("### Track your health journey and see improvements over time")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                history_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter user ID to view history")

                gr.Markdown("### 📊 Analysis Options")
                analysis_type = gr.Radio(
                    label="📈 View Type",
                    choices=[
                        "📊 Complete Progress Summary",
                        "📈 Weekly Trends Analysis",
                        "🎯 Target Achievement Report",
                        "📋 Raw Data Export",
                        "🏆 Achievement Milestones"
                    ],
                    value="📊 Complete Progress Summary",
                    info="Choose what type of analysis you want to see"
                )

                weeks_to_show = gr.Slider(
                    label="📅 Weeks to Include",
                    minimum=1,
                    maximum=12,
                    value=4,
                    step=1,
                    info="How many recent weeks to analyze"
                )

                with gr.Row():
                    view_history_btn = gr.Button("📊 View Progress", variant="primary", size="lg")
                    export_data_btn = gr.Button("💾 Export Data", variant="secondary")
                    delete_history_btn = gr.Button("🗑️ Clear History", variant="stop")



        # Store history components
        self.history_components = {
            'user_id': history_user_id,
            'analysis_type': analysis_type,
            'weeks_to_show': weeks_to_show,
            # 'output': history_output
        }

    def _format_preferences_display(self, preferences: dict) -> str:
        """Format preferences for nice display"""

        lines = ["### 🍽️ Your Dietary Profile:"]

        # Diet type
        if preferences.get('diet_type', 'No Preference') != "No Preference":
            lines.append(f"- **Diet:** {preferences['diet_type']}")

        # Allergies (most important!)
        if preferences.get('allergies'):
            allergy_list = ", ".join(preferences['allergies'])
            lines.append(f"- **⚠️ ALLERGIES:** {allergy_list}")

        if preferences.get('other_allergies'):
            lines.append(f"- **⚠️ OTHER RESTRICTIONS:** {preferences['other_allergies']}")

        if len(lines) == 1:  # Only the header
            lines.append("- No specific dietary preferences set")

        return "\n".join(lines)



    def _format_analysis_output_with_targets(self, week_data, recommendation, user_targets):
      """Format analysis output including custom targets"""

      daily_steps = week_data['total_steps'] / 7
      daily_water = week_data['water_liters'] / 7

      # Safely get avg_sleep and avg_mood with defaults
      avg_sleep = week_data.get('avg_sleep', 0)
      avg_mood = week_data.get('avg_mood', 0)

      # Compare against custom targets
      targets_comparison = f"""
  ### 🎯 Target Performance:
  - **Steps:** {week_data['total_steps']:,} / {user_targets['weekly_steps']:,} {'✅' if week_data['total_steps'] >= user_targets['weekly_steps'] else '⚠️'} ({((week_data['total_steps'] / user_targets['weekly_steps']) * 100):.0f}%)
  - **Zone Minutes:** {week_data['zone_minutes']} / {user_targets['weekly_zone_minutes']} {'✅' if week_data['zone_minutes'] >= user_targets['weekly_zone_minutes'] else '⚠️'} ({((week_data['zone_minutes'] / user_targets['weekly_zone_minutes']) * 100):.0f}%)
  - **Sleep:** {avg_sleep:.1f}h / {user_targets['daily_sleep_hours']}h {'✅' if avg_sleep >= user_targets['daily_sleep_hours'] else '⚠️'}
  - **Water:** {week_data['water_liters']:.1f}L / {user_targets['weekly_water_liters']}L {'✅' if week_data['water_liters'] >= user_targets['weekly_water_liters'] else '⚠️'}
"""

      preferences_display = self._format_preferences_display(week_data['preferences'])

      return f"""# 🏥 Personalized Wellbeing Analysis & Recommendations

  ## 👤 User: {week_data['user_id']} | 📅 Week: {week_data['week_start']}

  {preferences_display}

  {targets_comparison}

  ### 📊 Your Weekly Data Summary:
  **🏃‍♂️ Activity:**
  - **Steps:** {week_data['total_steps']:,} ({daily_steps:.0f}/day)
  - **Zone Minutes:** {week_data['zone_minutes']}
  - **Exercise Sessions:** {week_data['exercise_sessions']}

  **😴 Sleep & Mood:**
  - **Sleep:** {week_data['sleep_hours']} hours daily
  - **Mood:** {week_data['mood_scores']}/10 daily scores

  **🍎 Nutrition:**
  - 🥛 **Dairy:** {week_data['dairy_liters']:.1f}L | 💧 **Water:** {week_data['water_liters']:.1f}L ({daily_water:.1f}L/day)
  - 🍎 **Fruits:** {week_data['fruits_grams']:.0f}g | 🥬 **Vegetables:** {week_data['vegetables_grams']:.0f}g
  - 🫘 **Legumes:** {week_data['legumes_grams']:.0f}g | 🍗 **Meat:** {week_data['meat_grams']:.0f}g
  - 🌾 **Grains:** {week_data['grains_grams']:.0f}g | 🥜 **Nuts/Seeds:** {week_data['nuts_seeds_grams']:.0f}g

  ---

  ## 🤖 AI Recommendations
  *Personalized based on your data, preferences, allergies, and custom targets*

  {recommendation}

  ---

  💡 **Next Steps:**
  - **Food:** Follow dietary recommendations that match your preferences and restrictions
  - **Exercise:** Implement the suggested physical activities gradually
  - **Sleep & Social:** Focus on sleep hygiene and social connections for better wellbeing
  - **Overall:** Prioritize the main recommendation while maintaining your current strengths
  - **⚠️ Always verify ingredients** if you have allergies

  ✅ **Analysis Complete!** Your personalized recommendations have been saved for progress tracking.
  """

    def load_user_targets(self, user_id):
        """Load user targets and format for display in main tab"""
        try:
            if not user_id or not user_id.strip():
                return "❌ Please enter a valid User ID"

            targets = self.get_user_targets(user_id.strip())

            return f"""# 📊 Your Current Targets Loaded

## 🎯 Targets for {user_id}:

### 🏃‍♂️ Activity Targets:
- **Weekly Steps:** {targets['weekly_steps']:,}
- **Zone Minutes:** {targets['weekly_zone_minutes']}
- **Exercise Sessions:** {targets['weekly_exercise_sessions']}

### 😴 Recovery Target:
- **Daily Sleep:** {targets['daily_sleep_hours']} hours

### 🍎 Nutrition Targets:
- **Weekly Water:** {targets['weekly_water_liters']} liters
- **Weekly Fruits:** {targets['weekly_fruits_grams']:,} grams
- **Weekly Vegetables:** {targets['weekly_vegetables_grams']:,} grams

*These targets will be used to evaluate your progress when you get AI recommendations.*

**💡 Tip:** Go to the 'Customize Targets' tab to modify these values if needed.
"""

        except Exception as e:
            return f"❌ Error loading targets: {str(e)}"

    def save_user_targets(self, user_id, weekly_steps, zone_minutes, exercise_sessions,
                          daily_sleep, weekly_water):
        """Saves user's custom targets to a file"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to save targets."

        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        try:
            targets_data = {
                'weekly_steps': int(weekly_steps) if weekly_steps else self.default_targets['weekly_steps'],
                'weekly_zone_minutes': int(zone_minutes) if zone_minutes else self.default_targets['weekly_zone_minutes'],
                'weekly_exercise_sessions': int(exercise_sessions) if exercise_sessions else self.default_targets['weekly_exercise_sessions'],
                'daily_sleep_hours': float(daily_sleep) if daily_sleep else self.default_targets['daily_sleep_hours'],
                'weekly_water_liters': float(weekly_water) if weekly_water else self.default_targets['weekly_water_liters'],
              }

            with open(targets_file, 'w') as f:
                json.dump(targets_data, f, indent=2)

            return f"✅ Targets saved successfully for user '{user_id.strip()}'."

        except Exception as e:
            return f"❌ Error saving targets: {str(e)}"

    def get_user_targets(self, user_id: str) -> Dict:
        """Loads user's custom targets, or returns defaults"""
        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        if os.path.exists(targets_file):
            try:
                with open(targets_file, 'r') as f:
                    targets = json.load(f)
                    # Ensure all keys from default are present (for backward compatibility)
                    full_targets = self.default_targets.copy()
                    full_targets.update(targets)
                    return full_targets
            except Exception as e:
                print(f"⚠️ Warning: Could not load targets for user '{user_id.strip()}': {str(e)}. Using defaults.")
                return self.default_targets
        else:
            return self.default_targets

    def load_user_targets_display(self, user_id):
        """Loads user targets and formats for display in the targets tab"""
        if not user_id or not user_id.strip():
             return "❌ Error: Please enter a valid User ID to load targets.", *[gr.Number(value=self.default_targets[key]) for key in self.default_targets] # Return defaults to update UI

        targets = self.get_user_targets(user_id.strip())

        output_message = f"✅ Loaded targets for user '{user_id.strip()}'."

        # Return the message and the target values to update the Gradio Number components
        return (
            output_message,
            targets.get('weekly_steps', self.default_targets['weekly_steps']),
            targets.get('weekly_zone_minutes', self.default_targets['weekly_zone_minutes']),
            targets.get('weekly_exercise_sessions', self.default_targets['weekly_exercise_sessions']),
            targets.get('daily_sleep_hours', self.default_targets['daily_sleep_hours']),
            targets.get('weekly_water_liters', self.default_targets['weekly_water_liters']),
            )

    def reset_to_defaults(self):
        """Resets the target input fields to default values"""
        return (
            "🎯 Resetting targets to defaults.",
            self.default_targets['weekly_steps'],
            self.default_targets['weekly_zone_minutes'],
            self.default_targets['weekly_exercise_sessions'],
            self.default_targets['daily_sleep_hours'],
            self.default_targets['weekly_water_liters'],
        )

    def view_user_history(self, user_id, analysis_type, weeks_to_show):
        """Displays user history based on selected analysis type"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to view history."

        history = self.wellbeing_system.data_manager.get_user_history(user_id.strip())

        if not history:
            return f"📊 No history found for user '{user_id.strip()}'."

        # Limit history to weeks_to_show
        recent_history = history[-weeks_to_show:]

        output = f"## 📈 History for User: {user_id.strip()}\n"
        output += f"📊 **Analysis Type:** {analysis_type}\n"
        output += f"📅 **Weeks Included:** {len(recent_history)}\n\n"

        if analysis_type == "📊 Complete Progress Summary":
            output += "### Weekly Summary (Most Recent First)\n"
            for week in reversed(recent_history):
                output += f"#### Week Start: {week.get('week_start', 'Unknown')}\n"
                output += f"- Steps: {week.get('total_steps', 0):,}\n"
                output += f"- Zone Minutes: {week.get('zone_minutes', 0)}\n"
                output += f"- Sleep (Avg): {week.get('avg_sleep', 0):.1f}h\n"
                output += f"- Mood (Avg): {week.get('avg_mood', 0):.1f}/10\n"
                output += f"- Exercise Sessions: {week.get('exercise_sessions', 0)}\n"
                # Add a snippet of the recommendation if available
                output += f"- Recommendation Snippet: {week.get('recommendation', 'N/A')[:100]}...\n\n"

        elif analysis_type == "📈 Weekly Trends Analysis":
             # This would ideally involve plotting, but we'll provide text summary
            progress = self.wellbeing_system.get_user_progress(user_id.strip())
            output += "### Trends (Last 2 Weeks vs Before)\n"
            if 'message' in progress:
                 output += progress['message']
            else:
                output += f"- Steps Change (Last Week vs Previous): {progress.get('steps_change', 0):+,}\n"
                output += f"- Zone Minutes Change: {progress.get('zone_change', 0):+,}\n"
                output += f"- Sleep Change: {progress.get('sleep_change', 0):+.1f}h\n"
                output += f"- Mood Change: {progress.get('mood_change', 0):+.1f}/10\n"
                if 'trends' in progress:
                    output += "\n### Longer Term Trends (Last 4 Weeks Avg)\n"
                    output += f"- Avg Steps: {progress['trends'].get('avg_steps_4weeks', 0):,.0f}\n"
                    output += f"- Avg Sleep: {progress['trends'].get('avg_sleep_4weeks', 0):.1f}h\n"
                    output += f"- Consistency Score (Steps): {progress['trends'].get('consistency_score', 0):.1f}/10\n"


        elif analysis_type == "🎯 Target Achievement Report":
            user_targets = self.get_user_targets(user_id.strip())
            output += "### Target Achievement (Based on Latest Week)\n"
            latest_week = recent_history[-1] if recent_history else {}

            if latest_week:
                 output += f"- Weekly Steps: {latest_week.get('total_steps', 0):,} / {user_targets['weekly_steps']:,} ({((latest_week.get('total_steps', 0) / user_targets['weekly_steps']) * 100):.0f}%) {'✅' if latest_week.get('total_steps', 0) >= user_targets['weekly_steps'] else '⚠️'}\n"
                 output += f"- Weekly Zone Minutes: {latest_week.get('zone_minutes', 0)} / {user_targets['weekly_zone_minutes']} ({((latest_week.get('zone_minutes', 0) / user_targets['weekly_zone_minutes']) * 100):.0f}%) {'✅' if latest_week.get('zone_minutes', 0) >= user_targets['weekly_zone_minutes'] else '⚠️'}\n"
                 output += f"- Daily Sleep (Avg): {latest_week.get('avg_sleep', 0):.1f}h / {user_targets['daily_sleep_hours']}h {'✅' if latest_week.get('avg_sleep', 0) >= user_targets['daily_sleep_hours'] else '⚠️'}\n"
                 output += f"- Weekly Water (Avg): {latest_week.get('water_liters', 0):.1f}L / {user_targets['weekly_water_liters']}L {'✅' if latest_week.get('water_liters', 0) >= user_targets['weekly_water_liters'] else '⚠️'}\n" # Assuming water is stored with units in processed data
                 output += f"- Weekly Fruits (Avg): {latest_week.get('fruits_grams', 0):.0f}g / {user_targets['weekly_fruits_grams']:.0f}g {'✅' if latest_week.get('fruits_grams', 0) >= user_targets['weekly_fruits_grams'] else '⚠️'}\n"
                 output += f"- Weekly Vegetables (Avg): {latest_week.get('vegetables_grams', 0):.0f}g / {user_targets['weekly_vegetables_grams']:.0f}g {'✅' if latest_week.get('vegetables_grams', 0) >= user_targets['weekly_vegetables_grams'] else '⚠️'}\n"
                 output += f"- Weekly Exercise Sessions: {latest_week.get('exercise_sessions', 0)} / {user_targets['weekly_exercise_sessions']} {'✅' if latest_week.get('exercise_sessions', 0) >= user_targets['weekly_exercise_sessions'] else '⚠️'}\n"

            else:
                output += "No recent data to compare against targets."


        elif analysis_type == "📋 Raw Data Export":
            output += "### Raw Data (JSON Format)\n"
            output += "```json\n"
            output += json.dumps(recent_history, indent=2)
            output += "\n```\n"

        elif analysis_type == "🏆 Achievement Milestones":
            output += "### Achievements (Conceptual - requires more logic)\n"
            output += "This feature is under development. Here are some potential milestones:\n"
            output += "- First week hitting all targets!\n"
            output += "- Longest streak of 10k+ steps daily.\n"
            output += "- Personal best for zone minutes.\n"
            output += "- Consistent 8 hours of sleep for a month.\n"


        return output

    def export_user_data(self, user_id):
        """Exports all user data to a JSON file"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to export data."

        history = self.wellbeing_system.data_manager.get_user_history(user_id.strip())
        recommendations_dir = f"{self.wellbeing_system.data_manager.data_dir}/recommendations"
        user_rec_files = [f for f in os.listdir(recommendations_dir) if f.startswith(f"{user_id.strip()}_") and f.endswith('.json')]

        all_data = {"history": history, "recommendations": []}

        for rec_file in user_rec_files:
            try:
                with open(f"{recommendations_dir}/{rec_file}", 'r') as f:
                    all_data["recommendations"].append(json.load(f))
            except Exception as e:
                print(f"⚠️ Warning: Could not load recommendation file {rec_file}: {str(e)}")


        export_filename = f"wellbeing_export_{user_id.strip()}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        export_filepath = f"{self.wellbeing_system.data_manager.data_dir}/{export_filename}"

        try:
            with open(export_filepath, 'w') as f:
                json.dump(all_data, f, indent=2)
            return f"✅ Data exported successfully to `{export_filepath}`."
        except Exception as e:
            return f"❌ Error exporting data: {str(e)}"

    def delete_user_history(self, user_id):
        """Deletes all history and recommendations for a user"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to delete history."

        user_file = f"{self.wellbeing_system.data_manager.data_dir}/users/{user_id.strip()}.json"
        recommendations_dir = f"{self.wellbeing_system.data_manager.data_dir}/recommendations"

        deleted_count = 0
        try:
            # Delete user history file
            if os.path.exists(user_file):
                os.remove(user_file)
                deleted_count += 1

            # Delete recommendation files
            user_rec_files = [f for f in os.listdir(recommendations_dir) if f.startswith(f"{user_id.strip()}_") and f.endswith('.json')]
            for rec_file in user_rec_files:
                try:
                    os.remove(f"{recommendations_dir}/{rec_file}")
                    deleted_count += 1
                except Exception as e:
                    print(f"⚠️ Warning: Could not delete recommendation file {rec_file}: {str(e)}")

            if deleted_count > 0:
                 return f"🗑️ Successfully deleted {deleted_count} files for user '{user_id.strip()}'."
            else:
                 return f"ℹ️ No data found for user '{user_id.strip()}'. Nothing to delete."

        except Exception as e:
            return f"❌ Error deleting history: {str(e)}"


    def comprehensive_system_test(self):
        """Runs a comprehensive test of the system components"""
        output = "## 🧪 Comprehensive System Test Results\n"

        try:
            # Test LLM loading and generation
            output += "### 🤖 LLM Test\n"
            llm_test_result = self.wellbeing_system.llm.test_model()
            output += f"- Status: {llm_test_result}\n\n"

            # Test Data Management (basic save/load cycle)
            output += "### 💾 Data Management Test\n"
            test_user = "test_user_system"
            test_week = {
                'week_start': '2024-07-01',
                'total_steps': 75000,
                'zone_minutes': 160,
                'exercise_sessions': 4,
                'sleep_hours': '8,7.5,8,7,7.5,8,8',
                'mood_scores': '8,7,9,8,7,9,8',
                'dairy_liters': 3.0,
                'water_liters': 15.0,
                'fruits_grams': 2200,
                'vegetables_grams': 3600
            }
            test_rec = "Test recommendation output."

            try:
                self.wellbeing_system.data_manager.save_user_week(test_user, test_week)
                history = self.wellbeing_system.data_manager.get_user_history(test_user)
                output += f"- Save/Load History: {'✅ Success' if len(history) > 0 and history[-1].get('total_steps') == 75000 else '❌ Failed'}\n"

                self.wellbeing_system.data_manager.save_recommendation(test_user, test_week['week_start'], test_rec)
                last_rec = self.wellbeing_system.data_manager.get_last_recommendation(test_user)
                output += f"- Save/Load Recommendation: {'✅ Success' if last_rec == test_rec else '❌ Failed'}\n"

                # Clean up test data
                self.wellbeing_system.data_manager.delete_user_history(test_user) # This is now a method on data_manager
                cleanup_check = self.wellbeing_system.data_manager.get_user_history(test_user)
                output += f"- Cleanup: {'✅ Success' if not cleanup_check else '❌ Failed'}\n"

            except Exception as e:
                output += f"- Data Management Test Failed: {str(e)}\n"
                import traceback
                output += f"```\n{traceback.format_exc()}\n```\n"


            # Test Integration (analyze_and_recommend)
            output += "### 🔗 Integration Test\n"
            try:
                # Use sample data that includes preferences
                integration_test_data = {
                    'user_id': 'integration_user',
                    'week_start': '2024-07-08',
                    'total_steps': 50000,
                    'zone_minutes': 100,
                    'exercise_sessions': 2,
                    'sleep_hours': '6.5,7,6,7.5,7,6.8,7.2',
                    'mood_scores': '6,7,6,7,6,7,6',
                    'dairy_liters': 0.5, # Low dairy
                    'water_liters': 10.0, # Low water
                    'fruits_grams': 1500,
                    'vegetables_grams': 2500,
                    'grains_grams': 2000,
                    'nuts_seeds_grams': 150,
                    'meat_grams': 500,
                    'preferences': {
                         'diet_type': 'Vegetarian',
                         'allergies': ['Nuts (Tree nuts)'],
                         'other_allergies': 'Lactose Intolerant'
                    }
                }
                # Use default targets for this test
                integration_test_data['targets'] = self.default_targets.copy()


                integration_recommendation = self.wellbeing_system.analyze_and_recommend('integration_user', integration_test_data)

                output += f"- Analyze & Recommend: {'✅ Success' if len(integration_recommendation) > 100 and 'Vegetarian' in integration_recommendation and 'Nuts' not in integration_recommendation else '❌ Failed'}\n"
                output += f"  - Recommendation Length: {len(integration_recommendation)}\n"
                output += f"  - Recommendation Preview: {integration_recommendation[:150]}...\n"

                # Clean up integration test data
                self.wellbeing_system.data_manager.delete_user_history('integration_user')
                cleanup_check_int = self.wellbeing_system.data_manager.get_user_history('integration_user')
                output += f"- Cleanup: {'✅ Success' if not cleanup_check_int else '❌ Failed'}\n"


            except Exception as e:
                output += f"- Integration Test Failed: {str(e)}\n"
                import traceback
                output += f"```\n{traceback.format_exc()}\n```\n"


            output += "\n**🧪 System Test Complete.** Check the results above."

        except Exception as e:
            output += f"\n❌ An error occurred during the system test: {str(e)}"

        return output


    def check_llm_status(self):
        """Checks the status of the loaded LLM model"""
        if self.wellbeing_system.llm.model and self.wellbeing_system.llm.tokenizer:
            status = f"✅ LLM is loaded and ready.\n"
            status += f"- Base Model: {self.wellbeing_system.llm.model.config._name_or_path}\n"
            status += f"- Adapter: {self.wellbeing_system.llm.model.peft_config['default'].peft_adapter_id}\n"
            status += f"- Device: {self.wellbeing_system.llm.device}\n"
            try:
                 test_output = self.wellbeing_system.llm.test_model()
                 status += f"- Basic Test: {test_output}\n"
            except Exception as e:
                 status += f"- Basic Test Failed: {str(e)}\n"

        elif self.wellbeing_system.llm.model:
             status = "⚠️ LLM model is loaded, but tokenizer is missing.\n"
        elif self.wellbeing_system.llm.tokenizer:
             status = "⚠️ LLM tokenizer is loaded, but model is missing.\n"
        else:
            status = "❌ LLM model and tokenizer failed to load.\n"
            status += "Please check the model loading logs in the console for errors."
            status += "\n\n⚠️ **LLM Loading Issue:** The model might be too large for the available GPU RAM."
            status += "Consider trying a smaller model or a runtime with more GPU memory (e.g., A100)."


        return f"## 🤖 LLM System Status\n{status}"

    def check_storage_status(self):
        """Checks the status of the data storage directory"""
        data_dir = self.wellbeing_system.data_manager.data_dir
        output = f"## 💾 Data Storage Status\n"
        output += f"- Data Directory: `{data_dir}`\n"

        try:
            os.makedirs(data_dir, exist_ok=True)
            output += "- Directory exists or created: ✅\n"

            users_dir = f"{data_dir}/users"
            recs_dir = f"{data_dir}/recommendations"

            os.makedirs(users_dir, exist_ok=True)
            os.makedirs(recs_dir, exist_ok=True)

            output += f"- Users directory (`{users_dir}`) exists or created: ✅\n"
            output += f"- Recommendations directory (`{recs_dir}`) exists or created: ✅\n"

            # Count files
            user_files = [f for f in os.listdir(users_dir) if f.endswith('.json')]
            rec_files = [f for f in os.listdir(recs_dir) if f.endswith('.json')]

            output += f"- Number of user data files: {len(user_files)}\n"
            output += f"- Number of recommendation files: {len(rec_files)}\n"

            # Check if a dummy file can be written
            test_file = f"{data_dir}/test_write_{datetime.now().strftime('%Y%m%d%H%M%S')}.txt"
            try:
                with open(test_file, 'w') as f:
                    f.write("test")
                os.remove(test_file)
                output += "- Write permission: ✅\n"
            except Exception as e:
                output += f"- Write permission: ❌ ({str(e)})\n"

        except Exception as e:
            output += f"❌ An error occurred checking storage: {str(e)}\n"

        return output

    def analyze_with_agent_toggle(self, user_id, week_start, total_steps, zone_minutes, exercise_sessions,
                             sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
                             fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
                             diet_type, allergies, other_allergies):
      """Handle analysis with agent toggle support"""

      try:


          # Call your existing analysis method
          result = self.analyze_wellbeing_with_preferences(
              user_id, week_start, total_steps, zone_minutes, exercise_sessions,
              sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
              fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
              diet_type, allergies, other_allergies
          )

          return result

      except Exception as e:
          return f"❌ Error: {str(e)}"

# Main execution

In [ ]:
# Uncomment to run:
app = WellbeingApp(base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B", adapter_path="AnjaliNV/WellBeing_LLM")
demo = app.create_wellbeing_app()
demo.launch(share=True, debug=True)

# Fine Tuning

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import Dataset
import gc
import os
from typing import List, Dict

class ColabMemoryEfficientTrainer:
    def __init__(self, model_name: str):
        self.model_name = model_name

        # AGGRESSIVE memory cleanup
        self.clear_memory()

        print("🔧 Setting up ULTRA memory-efficient training for Google Colab...")

        # Set environment variables for memory optimization
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

        # Ultra-aggressive quantization
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=False,  # Disable for memory saving
        )

        print("📦 Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True,
            use_fast=True  # Use fast tokenizer for memory efficiency
        )

        print("🤖 Loading quantized model...")
        self.base_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            use_cache=False,  # Disable KV cache
            torch_dtype=torch.float16,
            device_map="auto"
        )

        # Prepare for k-bit training
        self.base_model = prepare_model_for_kbit_training(
            self.base_model,
            use_gradient_checkpointing=True  # Enable gradient checkpointing
        )

        # Setup ultra-minimal LoRA
        self.model = self.setup_minimal_lora()

        # Configure tokenizer
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        print("✅ Ultra memory-efficient setup complete!")

    def clear_memory(self):
        """Aggressive memory cleanup"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

    def setup_minimal_lora(self):
        """Setup MINIMAL LoRA for maximum memory efficiency"""

        lora_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            inference_mode=False,
            r=64,                      # MINIMAL rank for memory
            lora_alpha=256,             # Lower alpha
            lora_dropout=0.05,
            target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Only 2 modules to save memory
            bias="none",
        )

        model = get_peft_model(self.base_model, lora_config)
        model.train()
        model.print_trainable_parameters()

        return model

    def generate_minimal_training_examples(self):
        """Generate SMALL set of high-quality training examples"""

        examples = []

        # Only 8 carefully crafted examples for memory efficiency
        compact_examples = [
    {
        "input": """Analyze health data: Male, 28, 75kg, 180cm
**Activity:**
* **Steps:** 59,000 (8,429/day)
* **Zone Minutes:** 105
* **Exercise Sessions:** 2
**😴 Sleep & Mood:**
* **Sleep:** 7.8,7.5,8.0,7.2,7.5,8.2,7.8 hours daily
* **Mood:** 3,3,4,2,3,4,3/10 daily scores
**🍎 Nutrition:**
* 🥛 **Dairy:** 2.5L | 💧 **Water:** 35.0L (5.0L/day)
* 🍎 **Fruits:** 1800g | 🥬 **Vegetables:** 2800g
* 🫘 **Legumes:** 420g | 🍗 **Meat:** 0g
* 🌾 **Grains:** 2100g | 🥜 **Nuts/Seeds:** 180g
**Diet:** Vegetarian | **Allergies:** Gluten""",

        "output": """## 1) Food Recommendation
- Overall Assessment: Vegetarian diet with adequate legumes (420g) and vegetables (2800g) but low water intake (5.0L/day vs 8L target). Low mood (3.2/10) may relate to B12 or iron deficiency.
- Areas to Improve: -- Increase water intake from 5.0L to 8L daily for better hydration
                     -- Add iron-rich foods like spinach and lentils for energy
                     -- Include B12 sources like nutritional yeast for mood support
- Meal and Food Suggestions: -- Vegetables: Dark leafy greens, bell peppers, broccoli (target 4000g weekly)
                             -- Proteins: Lentils, quinoa, chickpeas, hemp seeds (increase legumes to 500g)
                             -- Grains: Gluten-free oats, quinoa, brown rice (maintain 2100g)

## 2) Physical Exercise Suggestion
- Activity and Performance Analysis: 59,000 weekly steps averaging 8,429 daily is good baseline, approaching 10,000 daily target.
- Zone Minutes and Intensity Feedback: 105 zone minutes is 45 minutes below 150 weekly target. Add 7 minutes daily.
- Strength Training Analysis: 2 sessions weekly meets minimum for muscle maintenance.
- Cardio Suggestions: Add 15 minutes daily brisk walking or cycling to reach 150 zone minutes.

## 3) Sleep and Social Well-being
- Sleep/Mood Analysis: Average 7.7 hours sleep is excellent, but mood averaging 3.2/10 needs attention despite good sleep.
- Suggestions: -- Maintain consistent 7.5+ hour sleep schedule
               -- Join vegetarian cooking groups for social connection and meal ideas
               -- Practice outdoor activities for mood-boosting vitamin D

## 4) Overall Weekly Recommendation
- Summary: Good sleep foundation (7.7h) and vegetarian nutrition base, but focus needed on hydration, mood support, and reaching activity targets.
- Weekly Goals: -- Increase water from 5.0L to 6.5L daily this week
                -- Add 45 zone minutes through 7 extra minutes daily activity
                -- Include iron-rich vegetables to support energy and mood"""
    },

    {
        "input": """Analyze health data: Female, 35, 65kg, 165cm
**Activity:**
* **Steps:** 42,000 (6,000/day)
* **Zone Minutes:** 85
* **Exercise Sessions:** 1
**😴 Sleep & Mood:**
* **Sleep:** 6.5,6.0,7.0,6.5,6.0,7.5,6.8 hours daily
* **Mood:** 4,4,5,3,4,6,4/10 daily scores
**🍎 Nutrition:**
* 🥛 **Dairy:** 1.5L | 💧 **Water:** 28.0L (4.0L/day)
* 🍎 **Fruits:** 1400g | 🥬 **Vegetables:** 2100g
* 🫘 **Legumes:** 210g | 🍗 **Meat:** 560g
* 🌾 **Grains:** 1800g | 🥜 **Nuts/Seeds:** 140g
**Diet:** Keto | **Allergies:** Tree nuts""",

        "output": """## 1) Food Recommendation
- Overall Assessment: Keto diet with low carb intake (1800g grains is high for keto), good meat protein (560g), but critically low water (4.0L vs 8L target) and vegetables (2100g vs 3500g target).
- Areas to Improve: -- Dramatically increase water from 4.0L to 8L daily for keto kidney support
                     -- Reduce grains from 1800g to under 500g for proper ketosis
                     -- Increase vegetables from 2100g to 3000g for micronutrients
- Meal and Food Suggestions: -- Vegetables: Leafy greens, broccoli, cauliflower, zucchini (avoiding high-carb ones)
                             -- Proteins: Fatty fish, chicken thighs, eggs (maintain 560g meat)
                             -- Fats: Avocado, olive oil, coconut oil (avoid tree nuts due to allergy)

## 2) Physical Exercise Suggestion
- Activity and Performance Analysis: 42,000 weekly steps averaging 6,000 daily is below 10,000 target, indicating sedentary lifestyle.
- Zone Minutes and Intensity Feedback: 85 zone minutes significantly below 150 target, need 65 more minutes weekly (9 minutes daily).
- Strength Training Analysis: Only 1 session weekly insufficient. Increase to 2-3 sessions for metabolic benefits.
- Cardio Suggestions: Add 20-minute daily walks and one swimming session for low-impact cardio boost.

## 3) Sleep and Social Well-being
- Sleep/Mood Analysis: Average 6.6 hours sleep below 7-8 hour target. Mood averaging 4.3/10 shows room for improvement.
- Suggestions: -- Extend sleep to 7.5 hours with consistent bedtime routine
               -- Join keto support groups for community and meal planning
               -- Practice stress reduction as low sleep may affect mood

## 4) Overall Weekly Recommendation
- Summary: Keto diet needs refinement with proper carb restriction, critical hydration increase needed, plus more activity and sleep.
- Weekly Goals: -- Increase water from 4L to 6L daily immediately
                -- Add 30 minutes daily walking to reach 8,000 steps
                -- Reduce grain intake by 50% for proper ketosis"""
    },

    {
        "input": """Analyze health data: Male, 45, 85kg, 175cm
**Activity:**
* **Steps:** 78,000 (11,143/day)
* **Zone Minutes:** 180
* **Exercise Sessions:** 5
**😴 Sleep & Mood:**
* **Sleep:** 8.2,8.5,8.0,8.3,8.1,8.4,8.2 hours daily
* **Mood:** 5,4,5,4,5,4,5/10 daily scores
**🍎 Nutrition:**
* 🥛 **Dairy:** 4.2L | 💧 **Water:** 56.0L (8.0L/day)
* 🍎 **Fruits:** 2800g | 🥬 **Vegetables:** 4200g
* 🫘 **Legumes:** 490g | 🍗 **Meat:** 980g
* 🌾 **Grains:** 3200g | 🥜 **Nuts/Seeds:** 280g
**Diet:** No preference | **Allergies:** None""",

        "output": """## 1) Food Recommendation
- Overall Assessment: Excellent nutrition profile with high protein (980g meat + 490g legumes), optimal hydration (8.0L daily), and outstanding vegetable intake (4200g vs 3500g target).
- Areas to Improve: -- Monitor protein timing around 5 exercise sessions for optimal recovery
                     -- Consider anti-inflammatory foods for high training volume
                     -- Maintain current excellent intake levels
- Meal and Food Suggestions: -- Recovery foods: Berries, tart cherries, leafy greens for inflammation
                             -- Proteins: Continue lean meats, add fatty fish twice weekly
                             -- Carbs: Sweet potatoes, quinoa, oats for exercise fuel (maintain 3200g)

## 2) Physical Exercise Suggestion
- Activity and Performance Analysis: Outstanding 78,000 steps averaging 11,143 daily, far exceeding 10,000 target.
- Zone Minutes and Intensity Feedback: Excellent 180 zone minutes, well above 150 target showing high cardiovascular fitness.
- Strength Training Analysis: 5 sessions weekly is high volume, monitor for overtraining with current mood levels.
- Cardio Suggestions: Maintain current level but incorporate active recovery days to prevent burnout.

## 3) Sleep and Social Well-being
- Sleep/Mood Analysis: Excellent 8.2 hours average sleep supporting high activity. Mood at 4.6/10 could be higher given optimal lifestyle.
- Suggestions: -- Maintain current excellent sleep routine for recovery
               -- Join fitness groups or sports teams for social engagement
               -- Consider stress management as high training may affect mood

## 4) Overall Weekly Recommendation
- Summary: Exceptional fitness and nutrition profile with room for mood optimization and overtraining prevention.
- Weekly Goals: -- Schedule 1-2 complete rest days for mental recovery
                -- Add social fitness activities for mood enhancement
                -- Monitor energy levels to prevent overtraining while maintaining excellence"""
    }
]

        # Duplicate with variations to reach 8 examples
        for i, example in enumerate(compact_examples):
            examples.append(example)

            # Create slight variation
            if i < 2:  # Only vary first 2 to keep memory low
                varied_input = example["input"].replace("Steps:", "Weekly steps:").replace("Zone:", "Zone minutes:")
                examples.append({
                    "input": varied_input,
                    "output": example["output"]
                })

        print(f"📚 Generated {len(examples)} memory-efficient training examples")
        return examples

    def format_training_data(self, examples: List[Dict]):
        """Format with memory optimization"""
        formatted_data = []
        for example in examples:
            conversation = f"Human: {example['input']}\n\nAssistant: {example['output']}"
            formatted_data.append({"text": conversation})
        return formatted_data

    def tokenize_function(self, examples):
        """Tokenize with aggressive memory limits"""
        tokenized = self.tokenizer(
            examples["text"],
            truncation=True,
            padding=False,  # Dynamic padding to save memory
            max_length=512,  # Shorter sequences
        )
        tokenized["labels"] = tokenized["input_ids"].copy()
        return tokenized

    def train_with_memory_constraints(self, output_dir: str = "./colab_template_lora"):
        """Ultra memory-efficient training for Colab"""

        print("🎯 Creating memory-efficient training dataset...")

        examples = self.generate_minimal_training_examples()
        training_data = self.format_training_data(examples)

        print(f"📊 Training on {len(examples)} examples with memory optimization")

        # Create small dataset
        dataset = Dataset.from_list(training_data)

        # Clear memory before tokenization
        self.clear_memory()

        tokenized_dataset = dataset.map(
            self.tokenize_function,
            batched=True,
            batch_size=1,  # Process one at a time
            remove_columns=dataset.column_names
        )

        # Memory-optimized data collator
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,
            pad_to_multiple_of=8,  # Efficient padding
        )

        # ULTRA memory-efficient training arguments (fixed for newer transformers)
        training_args = TrainingArguments(
            output_dir=output_dir,
            overwrite_output_dir=True,
            num_train_epochs=30,                # More epochs to compensate for small dataset
            per_device_train_batch_size=1,      # Smallest possible batch
            gradient_accumulation_steps=4,      # Smaller accumulation
            warmup_steps=10,
            weight_decay=0.01,
            logging_steps=1,
            save_strategy="no",                 # Don't save during training to save memory
            learning_rate=5e-4,                 # Higher LR for faster learning
            fp16=True,
            gradient_checkpointing=True,        # Enable gradient checkpointing
            remove_unused_columns=True,
            dataloader_pin_memory=False,
            dataloader_num_workers=0,
            report_to=[],
            max_grad_norm=0.3,
            lr_scheduler_type="cosine",       # Simplest scheduler
            save_total_limit=1,
            prediction_loss_only=True,
        )

        # Clear memory before creating trainer
        self.clear_memory()

        # Create trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator,
        )

        print("🚀 Starting memory-efficient template training...")
        print("💾 Using minimal rank, small batch, gradient checkpointing!")

        try:
            # Train the model
            trainer.train()

            print("💾 Saving trained model...")
            self.model.save_pretrained(output_dir)
            self.tokenizer.save_pretrained(output_dir)

            print(f"✅ Memory-efficient model saved to {output_dir}")
            return self.model, self.tokenizer

        except torch.cuda.OutOfMemoryError as e:
            print(f"❌ Still out of memory: {e}")
            print("🔧 Try these options:")
            print("1. Restart Colab runtime and try again")
            print("2. Use Colab Pro for more GPU memory")
            print("3. Use the prompt-engineering solution instead")
            return None, None

    def test_quick(self):
        """Quick test with minimal memory usage"""
        test_input = "Health data: Male, 30, 70kg, 175cm, Steps: 50000, Zone: 100min, Sleep: 7h, Diet: Vegetarian"

        prompt = f"Human: {test_input}\n\nAssistant:"
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=256, truncation=True)

        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                inputs["input_ids"],
                max_length=inputs["input_ids"].shape[1] + 200,
                temperature=0.3,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.replace(prompt, "").strip()

        print("🧪 Quick test response:")
        print(response[:300] + "...")

        # Check basic structure
        sections = ["## 1) Food Recommendation", "## 2) Physical Exercise", "## 3) Sleep", "## 4) Overall"]
        found = sum(1 for s in sections if any(part in response for part in s.split()))
        print(f"📊 Template sections detected: {found}/4")

def main():
    model_name = "ContactDoctor/Bio-Medical-Llama-3-8B"

    print("🚀 COLAB MEMORY-EFFICIENT TEMPLATE TRAINING")
    print("🔧 Optimized for Google Colab's memory constraints")

    try:
        # Initialize memory-efficient trainer
        trainer = ColabMemoryEfficientTrainer(model_name)

        # Train with memory constraints
        model, tokenizer = trainer.train_with_memory_constraints()

        if model is not None:
            # Quick test
            trainer.test_quick()
            print("\n✅ Memory-efficient training complete!")
        else:
            print("❌ Training failed due to memory constraints")
            print("💡 Consider using prompt engineering instead of fine-tuning")

    except Exception as e:
        print(f"❌ Training failed: {e}")
        print("💡 Alternative: Use stronger prompting without fine-tuning")

if __name__ == "__main__":
    main()

# ALTERNATIVE: If fine-tuning still fails, use this prompt engineering approach
def create_strong_template_prompt():
    """Create a very strong prompt for template adherence without fine-tuning"""

    template_prompt = """You are a wellness coach AI. You MUST follow this EXACT format. Any deviation will be rejected.

MANDATORY RESPONSE FORMAT - COPY EXACTLY:

## 1) Food Recommendation
- Overall Assessment: [Analyze user's current nutrition status, intake levels, and dietary balance]
- Areas to Improve: -- [List 2-3 specific nutritional improvements needed]
                     -- [Each improvement on new line with specific guidance]
- Meal and Food Suggestions: -- [Category]: [Specific food recommendations]
                             -- [Category]: [Specific food recommendations]

## 2) Physical Exercise Suggestion
- Activity and Performance Analysis: [Analyze step count, daily movement, and current activity level]
- Zone Minutes and Intensity Feedback: [Assess zone minutes vs 150 weekly target, provide specific guidance]
- Strength Training Analysis: [Evaluate current strength training, recommend frequency/type]
- Cardio Suggestions: [Specific cardio activities and duration recommendations]

## 3) Sleep and Social Well-being
- Sleep/Mood Analysis: [Analyze sleep duration and mood scores, identify patterns]
- Suggestions: -- [Sleep improvement strategies with specific actions]
               -- [Social and mood enhancement activities]

## 4) Overall Weekly Recommendation
- Summary: [Summarize user's strengths and main focus areas based on all data]
- Weekly Goals: -- [Specific, measurable goal 1]
                -- [Specific, measurable goal 2]
                -- [Specific, measurable goal 3]

START YOUR RESPONSE WITH "## 1) Food Recommendation":"""

    return template_prompt

print("💡 If training fails, use the strong template prompt above in your _build_personalized_prompt function!")

In [ ]:
from huggingface_hub import upload_folder

def upload_trained_model():
    """Upload your trained model to replace the existing adapter"""

    print("📤 Uploading trained model to Hugging Face...")

    try:
        upload_folder(
            folder_path="./colab_template_lora",  # Your trained model
            repo_id="AnjaliNV/WellBeing_LLM",     # Your repository
            commit_message="Update with stronger template training",
            create_pr=False  # Direct push to main branch
        )

        print("✅ Upload successful!")
        print("🎯 Your repository now has the new trained adapter")

    except Exception as e:
        print(f"❌ Upload failed: {e}")

# Run it
upload_trained_model()